In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [78]:
df=pd.read_csv('test.csv')

In [79]:
df.drop(['id'],axis=1,inplace=True)

In [80]:
df.head()

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [81]:
df.shape

(11490, 2)

In [82]:
tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(df['article'])
input_sequences = tokenizer_inputs.texts_to_sequences(df['article'])

tokenizer_outputs = Tokenizer()
tokenizer_outputs.fit_on_texts(df['highlights'])
output_sequences = tokenizer_outputs.texts_to_sequences(df['highlights'])

In [84]:
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)
max_input_length #  maximum value in all article

1910

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [93]:
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
decoder_input_sequences = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')


In [96]:
decoder_target_sequences = np.zeros(decoder_input_sequences.shape)

In [100]:
decoder_target_sequences[:, :-1] = decoder_input_sequences[:, 1:]

In [101]:
# Define vocabulary sizes
num_encoder_tokens = len(tokenizer_inputs.word_index) + 1   # input tokens for embedding layer   i.e for all the words
num_decoder_tokens = len(tokenizer_outputs.word_index) + 1

In [102]:
max_encoder_seq_length = max_input_length
max_decoder_seq_length = max_output_length

In [111]:
# Define model parameters
latent_dim = 256  # Adjust as needed
batch_size = 1   # Adjust as needed
epochs = 10       # Adjust as needed


In [105]:
# Define the encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [106]:
# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [107]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [113]:
model.fit([encoder_input_sequences, decoder_input_sequences], decoder_target_sequences,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/10
9192/9192 [==============================] - 1877s 204ms/step - loss: 0.5529 - val_loss: 0.6574
Epoch 2/10
9192/9192 [==============================] - 1867s 203ms/step - loss: 0.4934 - val_loss: 0.6698
Epoch 3/10
9192/9192 [==============================] - 3379s 368ms/step - loss: 0.4394 - val_loss: 0.6859
Epoch 4/10
9192/9192 [==============================] - 2724s 296ms/step - loss: 0.3901 - val_loss: 0.7061
Epoch 5/10
9192/9192 [==============================] - 3279s 357ms/step - loss: 0.3473 - val_loss: 0.7285
Epoch 6/10
9192/9192 [==============================] - 3112s 339ms/step - loss: 0.3114 - val_loss: 0.7475
Epoch 7/10
9192/9192 [==============================] - 2935s 319ms/step - loss: 0.2808 - val_loss: 0.7681
Epoch 8/10
9192/9192 [==============================] - 2787s 303ms/step - loss: 0.2548 - val_loss: 0.7886
Epoch 9/10
9192/9192 [==============================] - 2424s 264ms/step - loss: 0.2327 - val_loss: 0.8073
Epoch 10/10
9192/9192 [==============

In [114]:
model.save('model.h5')

In [5]:
loaded_model = tf.keras.saving.load_model("model.h5")
loaded_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 256)            3517926   ['input_1[0][0]']             
                                                          4                                       
                                                                                                  
 embedding_3 (Embedding)     (None, None, 256)            1119436   ['input_2[0][0]']        

In [54]:
for x in range(df.shape[0]):
    article=df.iloc[x:x+1,0:1].values
    highlights=df.iloc[x:x+1,1:2].values
    tokenizer_inputs.fit_on_texts([article[0][0]])
    tokenizer_outputs.fit_on_texts([highlights[0][0]])
    if len(article[0][0].split())>max_input_length:
        max_input_length=len(article[0][0].split())
    if len(highlights[0][0].split())>max_output_length:
        max_output_length=len(highlights[0][0].split())
    



In [61]:
input_sequences = tokenizer_inputs.texts_to_sequences([a[0][0]])
output_sequences = tokenizer_outputs.texts_to_sequences([h[0][0]])


In [66]:
# max_input_length = max(len(seq) for seq in input_sequences)
# max_input_length
len(input_sequences[0])

830

In [72]:
tokenizer_inputs.texts_to_sequences(['across'])

[[271]]

In [ ]:
encoder_input_sequences = pad_sequences(input_sequences, maxlen=850, padding='post')
encoder_input_sequences

In [51]:
len(tokenizer_inputs.word_index)

59857

In [ ]:
a=df.iloc[2:3,0:1].values
a[0][0]

In [60]:
h=df.iloc[2:3,1:2].values
h[0][0]

"Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital .\nPolice say he would have been over legal drink-drive limit at time of crash .\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving ."

In [58]:
m=m.reshape(1,1)
len(m[0][0].split())

69

In [75]:
import tensorflow as tf
import numpy as np
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
# The model will take as input an integer matrix of size (batch,
# input_length), and the largest integer (i.e. word index) in the input
# should be no larger than 999 (vocabulary size).
# Now model.output_shape is (None, 10, 64), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(32, 10))
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)
output_array

1/1 [==============================] - 0s 50ms/step
(32, 10, 64)


array([[[-0.00196074, -0.02135772, -0.04362501, ..., -0.01652638,
          0.02154041,  0.02266935],
        [ 0.01611585, -0.04028884,  0.0321898 , ...,  0.02318347,
          0.03314524,  0.0041486 ],
        [ 0.01930294,  0.01204542,  0.02200768, ...,  0.02742699,
          0.0114092 ,  0.04346487],
        ...,
        [ 0.01347807, -0.03351621,  0.04307661, ...,  0.04735649,
          0.02516514,  0.02334478],
        [-0.0445575 ,  0.01160324, -0.03195496, ..., -0.03243134,
          0.00382978,  0.00582314],
        [ 0.03525546,  0.00190321, -0.01665533, ..., -0.00285423,
          0.02281702, -0.04005935]],

       [[ 0.04547021, -0.0146823 , -0.00892843, ..., -0.03490139,
         -0.03265649,  0.01320997],
        [ 0.01609809,  0.03688388, -0.04170477, ..., -0.00472521,
          0.03684527,  0.00789084],
        [-0.01424219,  0.03080088, -0.04676735, ...,  0.03842786,
         -0.00551232, -0.00326425],
        ...,
        [-0.01091368,  0.04848974,  0.00600308, ...,  

In [74]:
input_array.shape

(32, 10)